In [1]:
!python -m pip install --upgrade pip
%pip install gradio ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import json
import requests
import re

# Gradio 테마 설정
theme = gr.themes.Base(
    font=[gr.themes.GoogleFont('Noto Sans KR')],
    font_mono=[gr.themes.GoogleFont('JetBrains Mono'), 'ui-monospace', 'Consolas', 'monospace'],
)
set_darkmode = '''
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
'''

In [ ]:
ENDPOINT = 'https://6b039-lhr.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-05-01-preview'
API_KEY = '***'

SEARCH_ENDPOINT = 'https://lhzsearch.search.windows.net'
SEARCH_API_KEY = '***'
SEARCH_INDEX_NAME = 'lhz-skill-index'
SEARCH_SEMENTIC_CONFIGURATION = 'lhz-skill-sementic'

SYSTEM_INSTRUCTION = ("""
당신은 늙은 현자 '제레드 간'이야. 
당신의 별명은 '미럴레이크의 대마도사'이고, 세계에 대해 모르는 것이 없어.
지금은 '로그호라이즌 TRPG'의 룰북에 포함된 특기(스킬) 정보를 조회하는 것을 도와주는 서브 마스터 역할을 맡고 있어. 
특기(스킬)에 관한 질문이 들어오면 검색 결과로 나오는 관련 특기명(특기의 이름)과 그 특기의 정보를 적절히 표 형태로 출력하고 사용자의 질문에 답해줘. 
각 특기에 대한 인용 링크를 반드시 특기 이름 우측에 달아줘.

검색 결과가 없다면 더 자세한 검색을 돕기 위해 사용자에게 몇 가지 질문을 던져줘.

---

너는 점잖은 노인이야. 모두에게 반말을 사용할 만큼 나이가 많아.
젊은이들에게 가르침을 준다고 생각하고 말해.
특히, 아래의 말투를 참고해서 비슷한 말투를 사용해줘.

- 허허허, 이 늙은이에게 맡겨두게나.
- 그래, 나는 **제레드=간**이라고 하네. 무엇이 궁금한고?
""")

HISTORY_PREFIX = [
    {'role': 'system', 'content': SYSTEM_INSTRUCTION},
]
REQUEST_HEADER = {
    'Content-Type': 'application/json',
    'api-key': API_KEY
}


def get_body(history):
    return {
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": SEARCH_ENDPOINT,
                "index_name": SEARCH_INDEX_NAME,
                "semantic_configuration": SEARCH_SEMENTIC_CONFIGURATION,
                "query_type": "semantic",
                "fields_mapping": {
                    "content_fields_separator": "\n",
                    "content_fields": [
                        "skill_classification", "tags", "max_sr", "timing",
                        "roll", "target", "range", "cost", "limit", "description",
                        "flavor_text", "job_classification"
                    ],
                    "filepath_field": "required_job",
                    "title_field": "name",
                    "url_field": "id",
                    "vector_fields": []
                },
                "in_scope": True,
                "filter": None,
                "strictness": 3,
                "top_n_documents": 10,
                "authentication": {
                        "type": "api_key",
                        "key": SEARCH_API_KEY
                }
            }
        }],
        "messages": HISTORY_PREFIX + history,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096,
        "stop": None,
        "stream": False
    }


def get_response(history):
    response = requests.post(
        ENDPOINT, headers=REQUEST_HEADER, json=get_body(history))
    if response.status_code != 200:
        return {
            'role': 'assistant',
            'content': f'### ⚠️ 오류가 발생했습니다.\n```json\n{json.dumps(response.json(), ensure_ascii=False, indent=2)}'
        }
    response_message = response.json()['choices'][0]['message']
    print(f'{response_message}')
    return response_message

In [4]:
history = []

def replace_citations(text):
    def replace_match(match):
        doc_number = int(match.group(1))  # 'doc1'에서 '1' 추출
        return f'[*{doc_number}]'

    return re.sub(r'\[doc(\d+)]', replace_match, text)


def chat(content, history):
    if not content:
        return '', ''
    
    for message in history:
        for key in ['options', 'metadata']:
            if key in message:
                del message[key]
    
    message = {'role': 'user', 'content': content}
    response_message = get_response(history + [message])
    response = response_message['content']
    print(response_message)
    
    documents = '☑️ 검색을 사용하지 않았습니다.'
    if (context := response_message.get('context', None)) and context.get('citations'):
        response = replace_citations(response)
        
        lines = ['✅ 검색이 사용되었습니다.\n\n### 🔍 검색한 내용']
        
        context = response_message['context']
        intents = json.loads(context['intent'])
        for intent in intents:
            lines.append(f'- {intent}')
        
        lines.append('\n### 🔮 검색 결과')
        if context['citations']:
            for i, citation in enumerate(context['citations'], 1):
                lines.append(f'\\[*{i}] **{citation['title']}** - {citation['filepath'] or '공용'} 특기')
                lines.append(f'```\n{citation['content']}\n```')
        else:
            lines.append('검색 결과가 없습니다.')
        documents = '\n'.join(lines)
        
    return response, documents


with gr.Blocks(theme=theme) as demo:
    gr.Markdown('# 🧙‍♂️ 제레드=간\n미럴레이크의 대마도사 **제레드=간**이 당신의 마스터링을 돕고자 서브 마스터로서 참석했습니다!')
    with gr.Row():
        with gr.Column():
            gr.Markdown('### 📜 특기 검색 결과\n', height=30)
            documents = gr.Markdown('... 검색 결과가 여기에 표시됩니다 ...', height=640)

        with gr.Column():
            chat_interface = gr.ChatInterface(
                chat, 
                type='messages', 
                additional_outputs=[documents], 
                examples=[
                    '잘부탁드립니다, 선생님.', 
                    '이동 거리를 증가시켜주는 특기를 검색해주세요.',
                    '경감을 부여하는 특기를 검색해주세요.',
                    '마법 공격 중에서 대미지가 가장 큰 특기는 뭔가요?'],
            )
        
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
